# Inspiration
###### Destes dados podemos tirar diversos insights interessantes e a idéia é estimular a comunidade a explorá-los.

###### -Como torcedor, quando o time adversário perde um jogador logo nos animamos, o grito no estádio aumenta e a esperança de uma goleada logo se acende. Mas, nos números, será que isto é realmente verdade ou estamos sendo enganados pelo nosso senso comum?
###### -Qual a probabilidade de sair o segundo gol, do mesmo time, após a marcação do primeiro?
###### -Os gols aos últimos minutos são eventos extremamente raros?
###### -Um time que ganhou um jogo de virada aumenta a probabilidade de ganhar o próximo jogo? (mudança na motivação)

In [26]:
import pandas as pd
import numpy as np

In [27]:
dfeventos = pd.read_csv("events_paulista.csv")
dfstatus = pd.read_csv("stats_paulista.csv")
dfjogos = pd.read_csv("jogos_paulista.csv")

# LIMPEZA / TRANSFORMAÇÃO 

In [28]:
colunasjogos = dfjogos.columns.to_list()

In [29]:
dfjogos = dfjogos.drop(labels=[ 
    'eventStatus', 'eventLeague','underOver', 'predGoalsHome', 
    'predGoalsAway', 'advice', 'formHome', 
    'formAway', 'attHome', 'attAway', 'defHome', 
    'defAway', 'poissonHome', 'poissonAway', 'h2hHome', 
    'h2hAway', 'goalsHome', 'goalsAway', 'totalHome', 
    'totalAway', 'homeOdd', 'drawOdd', 'awayOdd', 'over05', 
    'over05Ht', 'under15', 'under15Ht','date','hour','winner',
    'winOrDraw','logoAwayTeam','logoHomeTeam'],axis=1) 
#tira colunas inuteis do dfjogos. (colunas de dados de apostas)

In [30]:
dfstatus = dfstatus.replace(np.nan,0.0) #renomeia Nan para 0.0
dfjogos = dfjogos.replace(np.nan,0.0)
dfeventos = dfeventos.replace(np.nan,0.0)

In [31]:
dfeventos.head()

,idEvent,timeElapsed,idTeam,strTeam,idPlayer,strPlayer,tipo,detail,comments
0,317952,32,7826,Ferroviária,15278,Carlão,Card,Yellow Card,0.0
1,317952,40,7826,Ferroviária,9457,Rayan,subst,Substitution 1,0.0
2,317952,55,10018,Água Santa,55298,Uederson,subst,Substitution 1,0.0
3,317952,59,7826,Ferroviária,80600,Tony,Card,Yellow Card,0.0
4,317952,65,10018,Água Santa,9852,Felipe Azevedo,subst,Substitution 2,0.0


#### time vencedor , perdedor e empate
#cria coluna vencedor , perdedor e empate com nome do time

In [32]:
lista = [
    (dfjogos["intHomeScore"] > dfjogos["intAwayScore"]),
    (dfjogos["intHomeScore"] < dfjogos["intAwayScore"])
]
opcaovitoria = [dfjogos["strHomeTeam"], dfjogos["strAwayTeam"]]
dfjogos["vencedor"] = np.select(lista, opcaovitoria, default = None)

opcaoderrota = [dfjogos["strAwayTeam"], dfjogos["strHomeTeam"]]
dfjogos["perdedor"] = np.select(lista, opcaoderrota, default = None)

lista1 = [(dfjogos["intHomeScore"]==dfjogos["intAwayScore"])]
opcaoempate = ["empate"]

dfjogos["empate"] = np.select(lista1, opcaoempate ,default = None)

# np.select() usa a condição da lista para preencher a coluna criada com os dados passados na opção

In [33]:
dfjogos[dfjogos["idEvent"] == 317954] # esse é um exemplo dos jogos que teve cartao vermelho.
# loc usando condicao dentro do colchetes

,idEvent,dateEvent,eventCity,eventElapsed,eventRound,idHomeTeam,strHomeTeam,idAwayTeam,strAwayTeam,intHomeScore,intAwayScore,intHomeHt,intAwayHt,idWinner,vencedor,perdedor,empate
25,317954,2020-02-22 15:00:00-03:00,"São Paulo, São Paulo",90,Regular Season - 7,10018,Água Santa,131,Corinthians,2,1,1,1,0.0,Água Santa,Corinthians,None


#### número de vitorias do time

In [34]:
dfjogos.head(10)

,idEvent,dateEvent,eventCity,eventElapsed,eventRound,idHomeTeam,strHomeTeam,idAwayTeam,strAwayTeam,intHomeScore,intAwayScore,intHomeHt,intAwayHt,idWinner,vencedor,perdedor,empate
0,317888,2020-02-02 19:00:00-03:00,"Campinas, São Paulo",90,Regular Season - 4,138,Guarani Campinas,10003,Santo André,2,1,1,1,10003.0,Guarani Campinas,Santo André,None
1,317890,2020-02-03 20:00:00-03:00,"Ribeirão Preto, São Paulo",90,Regular Season - 4,2618,Botafogo SP,7848,Mirassol,0,6,0,3,7848.0,Mirassol,Botafogo SP,None
2,317891,2020-02-03 20:00:00-03:00,"São Paulo, São Paulo",90,Regular Season - 4,126,Sao Paulo,7834,Novorizontino,1,1,0,0,7834.0,None,None,empate
3,317892,2020-02-07 19:15:00-03:00,"Barueri, São Paulo",90,Regular Season - 5,143,Oeste,138,Guarani Campinas,1,1,1,1,138.0,None,None,empate
4,317907,2020-02-21 16:00:00-03:00,"Ribeirão Preto, São Paulo",90,Regular Season - 7,2618,Botafogo SP,1201,Inter De Limeira,2,1,0,1,1201.0,Botafogo SP,Inter De Limeira,None
5,317893,2020-02-07 21:30:00-03:00,"Mirassol, São Paulo",90,Regular Season - 5,7848,Mirassol,794,Bragantino,2,0,1,0,7848.0,Mirassol,Bragantino,None
6,654982,2021-03-14 19:00:00-03:00,"Bragança Paulista, São Paulo",90,Regular Season - 4,794,Bragantino,10003,Santo André,2,1,1,0,10003.0,Bragantino,Santo André,None
7,655032,2021-05-02 20:00:00-03:00,"São Paulo, São Paulo",90,Regular Season - 10,10003,Santo André,121,Palmeiras,0,1,0,1,121.0,Palmeiras,Santo André,None
8,317949,2020-01-25 15:00:00-03:00,"São Paulo, São Paulo",90,Regular Season - 2,10018,Água Santa,7834,Novorizontino,0,2,0,1,7834.0,Novorizontino,Água Santa,None
9,317894,2020-02-08 19:30:00-03:00,"Campinas, São Paulo",90,Regular Season - 5,139,Ponte Preta,121,Palmeiras,0,1,0,1,121.0,Palmeiras,Ponte Preta,None


In [35]:
dfj = dfjogos.groupby(by = 'strHomeTeam').nunique()
dfj['X'] = dfj['idAwayTeam'] != dfj['strAwayTeam'] 
dfj['S'] = dfj['vencedor'] + dfj['perdedor'] + dfj['empate']
dfj['B'] = dfj['dateEvent'] != dfj['S']

In [36]:
dfj[[ 'idEvent','S', 'vencedor', 'perdedor', 'empate','B']]

,idEvent,S,vencedor,perdedor,empate,B
strHomeTeam,,,,,,
Botafogo SP,14,10,4,5,1,True
Bragantino,17,15,4,10,1,True
Corinthians,18,14,4,9,1,True
Ferroviária,15,11,4,6,1,True
Guarani Campinas,15,12,6,5,1,True
Inter De Limeira,15,15,6,8,1,False
Ituano,16,15,7,7,1,True
Mirassol,16,13,4,8,1,True
Novorizontino,17,12,3,8,1,True


#### cartão vermelho 
#cria dataframe apenas com jogos que tiveram cartoes vermelhos.

In [37]:
vermelho = dfeventos[dfeventos['detail'] == "Red Card"]
vermelho.reset_index(inplace = True)

In [38]:
"""Ao renumerar os indices foi criado uma coluna index indesejada.
(a referencia esta sendo feita pela coluna idEvent) """ 
vermelho.pop('index') 

0       40
1      279
2      361
3      404
4      407
5      605
6      763
7      786
8      836
9      962
10    1028
11    1107
12    1137
13    1155
14    1311
15    1333
16    1338
17    1365
18    1426
19    1616
20    1617
21    1638
22    1666
23    1756
24    1916
25    2051
26    2393
27    2447
28    2576
29    2658
30    2682
31    2773
32    2890
33    2892
34    2966
35    3004
36    3075
37    3157
38    3298
39    3306
40    3311
41    3368
42    3563
Name: index, dtype: int64

In [39]:
oldname = vermelho.columns.to_list()

In [40]:
#renomeando as colunas no datafreme vermelho
newname  = ['idEvent', 'timeElapsedRC', 'idTeamRC', 'strTeamRC', 'idPlayerRC', 'strPlayerRC', 'tipoRC', 'detailRC', 'commentsRC']          
vermelho.columns = newname

"""oldname = ['idEvent', 'timeElapsed', 'idTeam', 'strTeam', 'idPlayer', 'strPlayer', 'tipo', 'detail', 'comments']                                    
zipped = zip(oldname ,newname)
trocar = dict(zipped)
vermelho.rename(columns = trocar, inplace = True)"""
#Essa segunda forma de resolcer o problema foi pensada no exemplo encontrado no livro:
#Python para analise de dados. o'reilly. (funcionou mas deixa um aviso "chained indexing", 
#acontece quando muda o dado apos um slice, no meu caso apos selecionar os labels)

"oldname = ['idEvent', 'timeElapsed', 'idTeam', 'strTeam', 'idPlayer', 'strPlayer', 'tipo', 'detail', 'comments']                                    \nzipped = zip(oldname ,newname)\ntrocar = dict(zipped)\nvermelho.rename(columns = trocar, inplace = True)"

In [41]:
vermelho.loc[vermelho['strTeamRC'] == 'Santos']   # EXEMPLO Santos teve 5 RC

,idEvent,timeElapsedRC,idTeamRC,strTeamRC,idPlayerRC,strPlayerRC,tipoRC,detailRC,commentsRC
2,574013,43,128,Santos,10506,Marinho,Card,Red Card,0.0
12,654998,61,128,Santos,9998,Guilherme Nunes,Card,Red Card,Violent conduct
18,655015,43,128,Santos,179664,Wellington Tim,Card,Red Card,Foul
24,814334,45,128,Santos,278495,Gabriel Pirani,Card,Red Card,Foul
40,317930,45,128,Santos,10002,Jobson,Card,Red Card,0.0


##### Criando coluna quantidade de cartoes vermelhos por time

In [42]:
dictRC = vermelho.groupby('strTeamRC').count().to_dict()
dictRC['quantRC'] = dictRC['idTeamRC']
dictRC = dictRC['quantRC']

In [43]:
 dfquantRC = pd.DataFrame(dictRC.values(),dictRC.keys(),dtype ="int64")

In [44]:
dfquantRC.reset_index(inplace = True)
dfquantRC.columns = ['strTeamRC','quantRC']

In [45]:
dfquantRC

,strTeamRC,quantRC
0,Botafogo SP,2
1,Bragantino,2
2,Corinthians,4
3,Ferroviária,3
4,Guarani Campinas,4
5,Inter De Limeira,3
6,Ituano,3
7,Mirassol,3
8,Novorizontino,1
9,Oeste,1


##### Criando colunas quantidade de vitorias / derrotas por times

In [46]:
dfjogos.columns

Index(['idEvent', 'dateEvent', 'eventCity', 'eventElapsed', 'eventRound',
       'idHomeTeam', 'strHomeTeam', 'idAwayTeam', 'strAwayTeam',
       'intHomeScore', 'intAwayScore', 'intHomeHt', 'intAwayHt', 'idWinner',
       'vencedor', 'perdedor', 'empate'],
      dtype='object')

In [47]:
dictvit = dfjogos.groupby('vencedor').count().to_dict()     #quantidade de vitorias
dictvit['quantVIT'] = dictvit['idEvent']
dictvit = dictvit['quantVIT']

In [48]:
 dfquantvit = pd.DataFrame(dictvit.values(),dictvit.keys(),dtype ="int64")

In [49]:
dfquantvit.reset_index(inplace = True)
dfquantvit.columns = ['strTeam','quantVIT']

In [50]:
dfquantvit

,strTeam,quantVIT
0,Botafogo SP,7
1,Bragantino,14
2,Corinthians,16
3,Ferroviária,10
4,Guarani Campinas,10
5,Inter De Limeira,12
6,Ituano,10
7,Mirassol,12
8,Novorizontino,11
9,Oeste,3


In [51]:
dictderrota = dfjogos.groupby('perdedor').count().to_dict()     #quantidade de derrotas
dictderrota['quantderrota'] = dictderrota['idEvent']
dictderrota = dictderrota['quantderrota']

In [52]:
 dfquantderrota = pd.DataFrame(dictderrota.values(),dictderrota.keys(),dtype ="int64")

In [53]:
dfquantderrota.reset_index(inplace = True)
dfquantderrota.columns = ['strTeam','quantderrota']

In [54]:
dfquantderrota

,strTeam,quantderrota
0,Botafogo SP,12
1,Bragantino,6
2,Corinthians,6
3,Ferroviária,8
4,Guarani Campinas,13
5,Inter De Limeira,15
6,Ituano,13
7,Mirassol,9
8,Novorizontino,9
9,Oeste,8


##### Criando colunas quantidade de empates por times

In [55]:
dfEMP = dfjogos[dfjogos['empate'] == 'empate']     #quantidade de empates
dfEMP = dfjogos.groupby('strHomeTeam').count()
dfEMP['quantEMP'] = dfEMP['idEvent']
dfEMP = dfEMP['quantEMP'].reset_index() 

In [56]:
dfEMP

,strHomeTeam,quantEMP
0,Botafogo SP,14
1,Bragantino,17
2,Corinthians,18
3,Ferroviária,15
4,Guarani Campinas,15
5,Inter De Limeira,15
6,Ituano,16
7,Mirassol,16
8,Novorizontino,17
9,Oeste,6


### Merge para criar informaçoes criadas acima QUANT RC ,VIT, E,DER


In [57]:
vermelho = vermelho.merge(dfquantRC, on = 'strTeamRC', how = 'left')
dfjogos = dfjogos.merge(dfquantvit, left_on = 'vencedor', right_on = 'strTeam')
dfjogos = dfjogos.merge(dfquantderrota, left_on = 'perdedor', right_on = 'strTeam')
dfjogos = dfjogos.merge(dfEMP, left_on = 'strHomeTeam', right_on = 'strHomeTeam')
#todas as informaçõe estao no dfjogos.

In [58]:
dfjogos_clounas = ['strTeam_x', 'strTeam_y'] 
dfjogos = dfjogos.drop(labels= dfjogos_clounas, axis = 1)

In [59]:
tabela = dfEMP.merge(dfquantderrota, left_on = 'strHomeTeam', right_on = 'strTeam')
tabela = tabela.merge(dfquantvit, left_on = 'strHomeTeam', right_on = 'strTeam', how = "outer"  )

In [60]:
tabela = tabela.drop( labels = 'strTeam_x',axis = 1)
tabela = tabela.drop( labels = 'strTeam_y',axis = 1)


### Unindo informaçoes

In [61]:
join1 = vermelho.merge(dfjogos, on = 'idEvent', how = 'left')

In [62]:
join2 = vermelho.merge(dfjogos, on = 'idEvent', how = 'right')

### Analise quanto o vermelho influencia no jogo

In [63]:
join1.groupby( join1['vencedor'] == join1['strTeamRC']).count()
# true é vitorias, False é derrotas

,idEvent,timeElapsedRC,idTeamRC,strTeamRC,idPlayerRC,strPlayerRC,tipoRC,detailRC,commentsRC,quantRC,...,intAwayScore,intHomeHt,intAwayHt,idWinner,vencedor,perdedor,empate,quantVIT,quantderrota,quantEMP
False,35,35,35,35,35,35,35,35,35,35,...,23,23,23,23,23,23,0,23,23,23
True,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,0,8,8,8


In [64]:
join1.groupby( join1['perdedor'] == join1['strTeamRC']).count()
# true é derrotas, False é vitorias

,idEvent,timeElapsedRC,idTeamRC,strTeamRC,idPlayerRC,strPlayerRC,tipoRC,detailRC,commentsRC,quantRC,...,intAwayScore,intHomeHt,intAwayHt,idWinner,vencedor,perdedor,empate,quantVIT,quantderrota,quantEMP
False,20,20,20,20,20,20,20,20,20,20,...,8,8,8,8,8,8,0,8,8,8
True,23,23,23,23,23,23,23,23,23,23,...,23,23,23,23,23,23,0,23,23,23


In [65]:
join1.groupby(join1['empate']).count()

,idEvent,timeElapsedRC,idTeamRC,strTeamRC,idPlayerRC,strPlayerRC,tipoRC,detailRC,commentsRC,quantRC,...,intHomeScore,intAwayScore,intHomeHt,intAwayHt,idWinner,vencedor,perdedor,quantVIT,quantderrota,quantEMP
empate,,,,,,,,,,,,,,,,,,,,,


##### ^feito quantos relação de cartao vermelho e resultado da partida

# Analisando o total de jogos RC e derrota

### Analise quanto o vermelho influencia no campeonato

##### Virada

In [41]:
a = dfeventos.loc[dfeventos['tipo'] == "Goal", 'timeElapsed']

In [42]:
dfeventos['primeiroGol'] = dfeventos.groupby('idEvent').evento_gol.transform('min')

AttributeError: 'DataFrameGroupBy' object has no attribute 'evento_gol'

In [ ]:
dfeventos['primeiroGolTime'] = dfeventos.loc[dfeventos['primeiroGol'] == dfeventos['evento_gol'],'strTeam']                       

In [ ]:
dfeventos.tail(60)

# analise geral

In [ ]:
X = dfstatus.groupby('strTeam').mean()

In [ ]:
X["mediapercentpasses"] = X["passes_accurate"] *100 / X["total_passes"].round(2)